In [ ]:
##### Create input JSON file for EQHazard

In [2]:
##### Main libraries/modules

import json, os, subprocess, shapely
import rasterio as rio
from rasterio.plot import show
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
# import pandas as pd

PROJ: proj_create_from_database: Cannot find proj.db


In [3]:
##### Function to calculate the epicentral and hypocentral distances using the haversine equation

def get_haversine_dist(r,lat1,long1,lat2,long2,z=0):
    r = 6371
    # Haversine
    d = 2*r*np.arcsin(np.sqrt(np.sin((lat2-lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((long2-long1)/2)**2))
    return d, np.sqrt(d**2 + z**2)

In [4]:
##### Import liquefaction susceptibility map for Bay Area
##### https://pubs.usgs.gov/of/2006/1037/
# df_susc = gpd.GeoDataFrame.from_file('C:\\Users\\BarryZheng\\Downloads\\of06-1037_4b.shp\\'+'sfq2py.shp')
df_liq_susc = gpd.GeoDataFrame.from_file(os.getcwd()+'\\of06-1037_4b.shp\\sfq2py.shp')
liq_susc_dict = {'VL': 'very low',
                 'L': 'low',
                 'M': 'low',
                 'H': 'low',
                 'L': 'low',
                 'W': 'water',
                 'NM': 'not mapped'}

In [5]:
##### Import depth to groundwater map, converted to WGS84 coordinate system and downsampled by a factor of 4
##### https://datadryad.org/stash/dataset/doi:10.6078/D1W01Q
ras_z2gw = rio.open(os.getcwd() + '\\MinDTWmodel_meters_wgs84_ds4.tif')
data_z2gw = ras_z2gw.read(1)

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,8))
show(ras_ls_susc, ax=ax1, title='liq_susc')
show(ras_z2gw.read(1), ax=ax2, transform=ras_z2gw.transform, cmap='Blues', title='depth to gw')
ax1.set_xlim([-122.7,-121.6])
ax1.set_ylim([37.3,38.3])
ax2.set_xlim([-122.7,-121.6])
ax2.set_ylim([37.3,38.3])
plt.show()

In [15]:
##### Input file name

fn_fp_list = ['fn', 'fp']
rup_ind_list = [0, 0, 0, 0]
# source_ind_list = [83, 108, 606, 626]
fn_fp_list = [None]
rup_ind_list = [0]
source_ind_list = [606]

In [22]:
[make_EQHazard_input(ii, rup_ind_list[jj], source_ind_list[jj]) for ii in fn_fp_list for jj in range(len(source_ind_list))]

C:/Users/BarryZheng/OneDrive - SlateGeotech/Fragility/output//sim_grid_ucerf3_0_606.json


[None]

In [21]:
def make_EQHazard_input(fn_fp, rup_ind, source_ind):
    
    ##### Other inputs
    flag_h_corr = 0
    flag_p_corr = 0
    flag_ucerf = 3
    flag_psha = 0
    prob_annual_exc = 0.05
    nSites = 5
    if fn_fp == 'fn':
    #     z2gw = np.hstack([np.random.rand(1)*(10-5)+5,np.random.rand(2)*(20-5)+5,np.random.rand(2)*(30-10)+10])
        z2gw = ([5,5,5,10,10])
    elif fn_fp == 'fp':
    #     z2gw = np.hstack([np.random.rand(3)*(7-3)+3,np.random.rand(2)*(10-5)+5])
        z2gw = ([3,3,3,5,5])

    if flag_psha == 1:
        str_rup = '_psha'
    else:
        str_rup =  '_' + str(rup_ind) + '_' + str(source_ind)

    # z2gw = np.linspace(10,5,nSites)

    
    ### Pick site type/definition
    # site_type = 'SingleLocation'
    site_type = 'Grid'
#     site_type = 'SiteList'
    
    if site_type == 'SiteList':
        # inName = os.getcwd() + '\\sim_s' + str(nSites) + '_' + flag_fn_fp + '_ucerf' + \
        #          str(flag_ucerf) + '_h' + str(flag_h_corr) + '_p' + str(flag_p_corr) + '.json'
        inName = os.getcwd() + '\\output\\' + '\\sim_s' + str(nSites) + '_' + fn_fp + '_ucerf' + \
                 str(flag_ucerf) + str_rup + '.json'
        inName = inName.replace('\\','/')
    elif site_type == 'Grid':
        inName = os.getcwd() + '\\output\\' + '\\sim_grid_ucerf' + \
                 str(flag_ucerf) + str_rup + '.json'
        inName = inName.replace('\\','/')
    
    
    ##### Basic
    out_dict = {}
    
    out_dict = {'Site': {},
                'EqRupture': {},
                'GMPE': {},
                'IntensityMeasure': {}}
    
    ##### Site
    out_dict['Site'].clear()

    ### Update if defining a single point
    # siteList = [37.8719, -122.2585, None] # site coordinate, Vs30 of point, if "None", then inferred from Wald and Allen (2008)

    ### Update if defining a grid
    lat_range = [37, 39, round((39-37)/0.01)] # min, max, nDiv
    long_range = [-123, -121, round((-121--123)/0.01)] # min, max, nDiv
    vs30 = None # Vs30 of grid, if "None", then inferred from Wald and Allen (2008)
    
    ### Update if defining a list of sites
    # pt1_line = [37.8745, -122.2542]
    # pt2_line = [37.8390, -122.2793]
    # pt1_line = [37.892, -122.277]
    # pt2_line = [37.808, -122.271]
    ###### pt1_fp = [37.822140, -122.236366]
    ###### pt2_fp = [37.901043, -122.297522]
    ###### pt1_fn = [37.871003, -122.253224]
    ###### pt2_fn = [37.851118, -122.289518]
    pt1_fp = [37.924836, -122.316215]
    pt2_fp = [37.802175, -122.218137]
    pt1_fn = [37.83284025, -122.2426565]
    pt2_fn = [37.93091825, -122.1199955]
    if fn_fp == 'fp':
        pt1_line = pt1_fp
        pt2_line = pt2_fp
    elif fn_fp == 'fn':
        pt1_line = pt1_fn
        pt2_line = pt2_fn
    nDiv_line = nSites

    
    ### Update if defining a list
    if site_type == 'SiteList':
        siteListMethod = 3 # 1 = provide list below, 2 = define in txt file and import, 3 = define from two points and divisions
        if siteListMethod == 1:
            siteList = [[37.8719, -122.2585, 240.0],
                        [37.86,   -122.2585, 750.0],
                        [37.85,   -122.2585, None]]
        elif siteListMethod == 2:
            siteListPath = 'siteList.txt'
            siteList = []
            with open(siteListPath, 'r') as f:
                for lines in f:
                    line = []
                    for i in lines.split(','):
                        if i.split():
                            if 'None' in i:
                                line.append(None)
                            else:
                                line.append(float(i))
                    siteList.append(line)
        elif siteListMethod == 3:
            siteList = []
            for i in range(nDiv_line):
                xc = pt1_line[0]+(i+0.5)*(pt2_line[0] - pt1_line[0])/nDiv_line
                yc = pt1_line[1]+(i+0.5)*(pt2_line[1] - pt1_line[1])/nDiv_line

                x1 = pt1_line[0]+(i+0.0)*(pt2_line[0] - pt1_line[0])/nDiv_line
                y1 = pt1_line[1]+(i+0.0)*(pt2_line[1] - pt1_line[1])/nDiv_line
                x2 = pt1_line[0]+(i+1.0)*(pt2_line[0] - pt1_line[0])/nDiv_line
                y2 = pt1_line[1]+(i+1.0)*(pt2_line[1] - pt1_line[1])/nDiv_line

                d = get_haversine_dist(0,np.radians(x1),np.radians(y1),
                                       np.radians(x2),np.radians(y2),z=0)

                siteList.append([xc,yc,vs30,d[0]])

            ## Liquefaction susceptibility
            liq_susc = []
            for i in range(nDiv_line):
                # Setting the coordinates for the point
                coord2search = shapely.geometry.Point((siteList[i][1],siteList[i][0])) # Longitude & Latitude
                # Searching for the geometry that intersects the point. Returning the index for the appropriate polygon.
                liq_susc.append(df_liq_susc[df_liq_susc.geometry.intersects(coord2search)].LIQ.values[0])
            liq_susc = [liq_susc_dict.get(i,None) for i in liq_susc]

            ## Landslide susceptibility
            ls_susc = [data_ls_susc[ras_ls_susc.index(i[1], i[0])] for i in siteList]
            ls_susc = [int(i) if i >= 0 else 0 for i in ls_susc]

    elif site_type == 'Grid':
        siteList = []
        for i in range(lat_range[2]+1):
            xc = lat_range[0]+i*(lat_range[1] - lat_range[0])/lat_range[2]
            for j in range(long_range[2]+1):
                yc = long_range[0]+j*(long_range[1] - long_range[0])/long_range[2]
                siteList.append([xc,yc,vs30])

    ### update dictionary
    if site_type == 'SingleLocation':
        site_loc = {'Latitude': coord[0],
                    'Longitude': coord[1],
                    'Vs30': vs30}
        out_dict['Site'].update({'Type': site_type,
                                 'Location': site_loc})
    elif site_type == 'Grid':
        site_loc = {'Latitude': {'Min': lat_range[0], 'Max': lat_range[1], 'Divisions': lat_range[2]},
                    'Longitude': {'Min': long_range[0], 'Max': long_range[1], 'Divisions': long_range[2]},
                    'Vs30': vs30}
        out_dict['Site'].update({'Type': site_type,
                                 site_type: site_loc})
    elif site_type == 'SiteList':
        site_loc_full = []
        for i in range(len(siteList)):
            site_loc = {'Location': {'Latitude': siteList[i][0],
                                     'Longitude': siteList[i][1]},
                        'Vs30': siteList[i][2],
                        'LSegment': siteList[i][3],
                        'LiqSusc': liq_susc[i],
                        'LsSusc': ls_susc[i],
                        'Z2gw': z2gw[i]}
            site_loc_full.append(site_loc)
        out_dict['Site'].update({'Type': site_type,
                                 site_type: site_loc_full})
      
    
    ##### EqRupture
    out_dict['EqRupture'].clear()

    ### define rupture
    # eq_rup_type = 'PointSource'
    eq_rup_type = 'ERF'

    ### actions for point source
    if eq_rup_type == 'PointSource':
        eq_mag = 7.05
        eq_rup_loc = {'Latitude': 37.9,
                      'Longitude': -122.3,
                      'Depth': 0.0}
        eq_avg_rake = 0.0
        eq_avg_dip = 90.0
        out_dict['EqRupture'].update({'Type': eq_rup_type,
                                      'Magnitude': eq_mag,
                                      'Location': eq_rup_loc,
                                      'AverageRake': eq_avg_rake,
                                      'AverageDip': eq_avg_dip})

    ### update dictionary
    elif eq_rup_type == 'ERF':
    #     eq_rup_forecast = 'WGCEP (2007) UCERF2 - Single Branch'
    #     eq_source_index = 28
    #     eq_rup_index = 7
        eq_rup_forecast = 'Mean UCERF3'
    #     eq_source_index = 606
    #     eq_rup_index = 0
    #     eq_prob = 2.0749991502810872E-8
        out_dict['EqRupture'].update({'Type': eq_rup_type,
                                      'RuptureForecast': eq_rup_forecast,
                                      'SourceIndex': source_ind,
                                      'RuptureIndex': rup_ind})
        
    ##### GMPE
    out_dict['GMPE'].clear()

    ### pick GMPE
    # gmpe_type = 'Campbell & Bozorgnia (2014)'
    # gmpe_type = 'Abrahamson, Silva & Kamai (2014)'
    gmpe_type = 'Boore, Stewart, Seyhan & Atkinson (2014)'

    ### define GMPE params
    gmpe_params = {}

    ### update dictionary
    out_dict['GMPE'].update({'Type': gmpe_type,
                             'Parameters': gmpe_params})
    
    ##### Intensity Measure

    out_dict['IntensityMeasure'].clear()

    ### choose type of IM to save: individual or all
    # im_type = 'SA'
    # im_type = 'PGA'
    im_type = 'PGV'
#     im_type = 'All'

    ### choose type of output to get: json, csv, geojson
    im_json_out = True
    im_csv_out = False
    im_geo_json_out = True

    ### define period for SA
    ### if unspecified, default period = [0.01, 0.02, 0.03, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 7.5, 10.0]
    im_period = [0.01, 0.05, 0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0]

    ### update dictionary
    if flag_psha == 1:
        im_type = 'UHS'
        out_dict['IntensityMeasure'].update({'Type': im_type,
                                             'ExceedenceProbability': prob_annual_exc,
                                             'EnableJsonOutput': im_json_out,
                                             'EnableCsvOutput': im_csv_out,
                                             'EnableGeoJsonOutput': im_geo_json_out})
    else:
        out_dict['IntensityMeasure'].update({'Type': im_type,
                                             'Periods': im_period,
                                             'EnableJsonOutput': im_json_out,
                                             'EnableCsvOutput': im_csv_out,
                                             'EnableGeoJsonOutput': im_geo_json_out})
        
    ##### Other Parameters
    # if flag_h_corr == 1:
    #     corr_h = True
    # else:
    #     corr_h = False

    # if flag_p_corr == 1:
    #     corr_p = True
    # else:
    #     corr_p = False

    # ### update dictionary
    # out_dict['Other'].update({'Corr_Dist': corr_h,
    #                           'Corr_Period': corr_p})
    
    
    ##### See output before saving
#     print(out_dict)
    
    
    #####
    print(inName)
    
    
    ##### save file
    with open(inName, 'w') as f:
        json.dump(out_dict, f, indent=4)
    f.close()